In [4]:
import pandas as pd
df = pd.read_excel('Vessel_Fuel_Consumption_&_CO2_Emissions.xlsx', sheet_name='SHIP_FUEL_EFFICIENCY')
print(df.head())
print(df.shape)


  SHIP_ID SHIP_TYPE_ID ROUTE_ID     MONTH  DISTANCE FUEL_TYPE_ID  \
0   NG001        TYPE1   ROUTE1   January    132.26   FUEL_TYPE1   
1   NG001        TYPE1   ROUTE2  February    128.52   FUEL_TYPE1   
2   NG001        TYPE1   ROUTE2     March     67.30   FUEL_TYPE1   
3   NG001        TYPE1   ROUTE2     April     71.68   FUEL_TYPE2   
4   NG001        TYPE1   ROUTE3       May    134.32   FUEL_TYPE1   

   FUEL_CONSUMPTION  CO2_EMISSIONS WEATHER_CONDITIONS  ENGINE_EFFICIENCY  
0           3779.77       10625.76             Stormy              92.14  
1           4461.44       12779.73           Moderate              92.98  
2           1867.73        5353.01               Calm              87.61  
3           2393.51        6506.52             Stormy              87.42  
4           4267.19       11617.03               Calm              85.61  
(1440, 10)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Handle categoricals (your data has IDs but also text columns)
df['SHIP_TYPE_ID'] = df['SHIP_TYPE_ID'].astype(str)
df['ROUTE_ID'] = df['ROUTE_ID'].astype(str)
df['FUEL_TYPE_ID'] = df['FUEL_TYPE_ID'].astype(str)

# Encode all categoricals
label_encoders = {}
for col in ['SHIP_TYPE_ID', 'ROUTE_ID', 'FUEL_TYPE_ID', 'MONTH', 'WEATHER_CONDITIONS']:
    le = LabelEncoder()
    df[f'{col}_NUM'] = le.fit_transform(df[col])
    label_encoders[col] = le

# Features (proven strongest from your data patterns)
features = ['SHIP_TYPE_ID_NUM', 'ROUTE_ID_NUM', 'MONTH_NUM', 'DISTANCE',
           'FUEL_TYPE_ID_NUM', 'WEATHER_CONDITIONS_NUM', 'ENGINE_EFFICIENCY']
X = df[features]
y_fuel = df['FUEL_CONSUMPTION']
y_co2 = df['CO2_EMISSIONS']

# Train-test split (save test indices for matching)
X_train, X_test, y_fuel_train, y_fuel_test, y_co2_train, y_co2_test = train_test_split(
    X,
    y_fuel,
    y_co2,
    test_size=0.2,
    random_state=42,
    stratify=df['SHIP_TYPE_ID']   # remove this argument if it gives an error
)



In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
import numpy as np

# Train models
fuel_model = RandomForestRegressor(n_estimators=100, random_state=42)
co2_model = RandomForestRegressor(n_estimators=100, random_state=42)

fuel_model.fit(X_train, y_fuel_train)
co2_model.fit(X_train, y_co2_train)

# Test predictions
fuel_pred = fuel_model.predict(X_test)
co2_pred = co2_model.predict(X_test)

# Quick accuracy check
print(f"Fuel R²: {r2_score(y_fuel_test, fuel_pred):.3f}")
print(f"CO2 R²: {r2_score(y_co2_test, co2_pred):.3f}")
print(f"Fuel MAE: {mean_absolute_error(y_fuel_test, fuel_pred):.0f}L")


Fuel R²: 0.959
CO2 R²: 0.948
Fuel MAE: 630L


In [7]:
# Get original rows corresponding to the test set
test_data = df.loc[X_test.index].reset_index(drop=True)

comparison_df = pd.DataFrame({
    'SHIP_ID': test_data['SHIP_ID'],
    'ROUTE_ID': test_data['ROUTE_ID'],
    'MONTH': test_data['MONTH'],
    'DISTANCE': test_data['DISTANCE'],
    'WEATHER_CONDITIONS': test_data['WEATHER_CONDITIONS'],
    'ENGINE_EFFICIENCY': test_data['ENGINE_EFFICIENCY'],
    'Actual_Fuel_L': y_fuel_test.values,
    'Predicted_Fuel_L': fuel_pred,
    'Fuel_Error_L': np.abs(y_fuel_test.values - fuel_pred),
    'Actual_CO2_kg': y_co2_test.values,
    'Predicted_CO2_kg': co2_pred,
    'CO2_Error_kg': np.abs(y_co2_test.values - co2_pred)
})

comparison_df.head(5)  # just to inspect first 5 rows


,SHIP_ID,ROUTE_ID,MONTH,DISTANCE,WEATHER_CONDITIONS,ENGINE_EFFICIENCY,Actual_Fuel_L,Predicted_Fuel_L,Fuel_Error_L,Actual_CO2_kg,Predicted_CO2_kg,CO2_Error_kg
0,NG036,ROUTE3,August,413.97,Moderate,80.29,18037.62,15706.1502,2331.4698,49624.50,43779.3222,5845.1778
1,NG020,ROUTE3,January,98.82,Moderate,79.87,2550.32,2968.8603,418.5403,6706.70,8304.2513,1597.5513
2,NG043,ROUTE1,April,22.13,Moderate,86.52,252.26,363.9383,111.6783,745.93,1070.8654,324.9354
3,NG022,ROUTE1,December,56.92,Stormy,87.56,1468.97,1622.1243,153.1543,3842.00,4463.6202,621.6202
4,NG076,ROUTE4,December,144.49,Moderate,72.20,3057.90,3918.1281,860.2281,7716.97,11408.9177,3691.9477


In [8]:
from google.colab import files

comparison_df.to_csv('vessel_ml_comparison.csv', index=False)
files.download('vessel_ml_comparison.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Example new voyages – you can change these later
new_voyages = pd.DataFrame({
    'SHIP_TYPE_ID_NUM': [3, 0, 3],  # example encoded values
    'ROUTE_ID_NUM': [0, 2, 1],
    'MONTH_NUM': [0, 5, 11],
    'DISTANCE': [200, 100, 350],
    'FUEL_TYPE_ID_NUM': [0, 1, 0],
    'WEATHER_CONDITIONS_NUM': [2, 0, 1],
    'ENGINE_EFFICIENCY': [85, 92, 78]
})

new_fuel_pred = fuel_model.predict(new_voyages)
new_co2_pred = co2_model.predict(new_voyages)

for i in range(len(new_voyages)):
    print(f"Row {i}: {new_fuel_pred[i]:.0f} L fuel, {new_co2_pred[i]:.0f} kg CO2")


Row 0: 7881 L fuel, 22483 kg CO2
Row 1: 2937 L fuel, 8418 kg CO2
Row 2: 14330 L fuel, 39086 kg CO2


In [10]:
import joblib

joblib.dump(fuel_model, 'fuel_model.pkl')
joblib.dump(co2_model, 'co2_model.pkl')
joblib.dump(label_encoders, 'encoders.pkl')


['encoders.pkl']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')